In [289]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sys import getsizeof
import json
from sqlalchemy import create_engine
import datetime as dt
from yelpapi import YelpAPI
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 0)

%matplotlib inline

In [323]:
api_key = ''
yelp_api = YelpAPI(api_key)

In [336]:
pd_ins_hist_2016 = pd.read_csv("../Data/Inspections_Slightly_Cleaned_2016.csv",low_memory=False,parse_dates=[12,16])
pd_ins_hist_2016.nunique()

businessname     4075
dbaname            34
legalowner       1772
namelast         3653
namefirst        1879
licenseno        4665
issdttm          4641
expdttm            10
licstatus           2
licensecat          4
descript            4
result             12
resultdttm      39184
violation         410
viollevel           4
violdesc          403
violdttm        39189
violstatus          2
statusdate      12174
comments        96411
address          3275
city               46
state               4
zip                36
property_id      2886
location         2377
Latitude         2264
Longitude        2282
dtype: int64

In [352]:
pd_ins_hist_2016.sort_values(by='violdttm',ignore_index=True,ascending=False,inplace=True)
# pd_ins_hist_2016.groupby(['result','violstatus'])[['violstatus']].count()

In [357]:
pd_ins_hist_2016.sample(5)

,businessname,dbaname,legalowner,namelast,namefirst,licenseno,issdttm,expdttm,licstatus,licensecat,descript,result,resultdttm,violation,viollevel,violdesc,violdttm,violstatus,statusdate,comments,address,city,state,zip,property_id,location,Latitude,Longitude
144274,100 Percent Delicia Food,NaN,BRENNAN PATRICK E,Marte,Civelis,87059,2013-04-05 12:47:23,2020-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2017-04-25 11:57:59,M-2-102.11,***,PIC Knowledge,2017-04-25 11:57:59,Fail,NaN,PIC will need to review with all staff proper ...,635 Hyde Park AVE,Roslindale,MA,02131,77476.0,"(42.278590000, -71.119440000)",42.278590000,-71.119440000
8510,Bay Sweets,NaN,SPRING ST PROPERTIES LLC,Bay Sweets LLC,Victor Leon Owner,353052,2018-01-30 15:07:12,2020-12-31 23:59:00,Active,RF,Retail Food,HE_Filed,2020-07-31 15:01:33,590.005/5-205.15-C,*,System Maintained in Good Repair (C),2020-07-31 15:01:33,Pass,2020-07-31 18:44:46,3 bay sink/provide 3 drain plugs to each sink ...,120 Spring ST,West Roxbury,MA,02132,129536.0,"(42.276500000, -71.165400000)",42.276500000,-71.165400000
19120,SUPER STOP & SHOP No. 459,NaN,SUPER STOP & SHOP ATTN,SUPER STOP & SHOP,Attn: Licensing Dept.,21934,2012-02-06 13:03:47,2020-12-31 23:59:00,Active,RF,Retail Food,HE_Pass,2020-02-03 10:25:42,590.004/4-302.13-PF,**,Temperature Measuring Devices Manual and Mech...,2020-02-03 10:25:42,Pass,2020-02-03 15:43:55,No irreversible thermometer for Hi-Temp dishwa...,460 BLUE HILL AV,ROXBURY,MA,02121,NaN,NaN,NaN,NaN
150145,Black Rose,NaN,ONE 60 STATE ASSOC MASS LPS,The Black Rose Inc.,,18686,2011-12-29 14:14:49,2020-12-31 23:59:00,Active,FS,Eating & Drinking,HE_FailExt,2017-03-06 11:02:41,16-4-501.11/.15,*,Dishwashng Facilities,2017-03-06 11:02:41,Pass,2017-03-06 12:55:23,Kitchen low temperature dishwasher not registe...,160 State ST,Boston,MA,02109,130383.0,"(42.359440000, -71.053790000)",42.359440000,-71.053790000
142152,THE BASEBALL TAVERN,NaN,NaN,JAMES ROONEY,,24495,2012-01-04 18:55:55,2019-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Fail,2017-05-10 10:33:49,25-4-904.11,*,Single Service Articles Stored Dispensed,2017-05-10 10:33:49,Fail,NaN,Bar areas: Straws are not individually wrapped...,1270 BOYLSTON,BOSTON,MA,02215,NaN,NaN,NaN,NaN


In [244]:
#Getting just one row of per restaurant & their respective details

df_restaurants = pd_ins_hist_2016.drop_duplicates(['businessname','licenseno'],ignore_index=True)

In [245]:
df_restaurants.licensecat.value_counts()

FS     2051
FT     1633
RF      838
MFW     143
Name: licensecat, dtype: int64

In [246]:
df_restaurants.groupby('licensecat')['descript'].nunique()

licensecat
FS     1
FT     1
MFW    1
RF     1
Name: descript, dtype: int64

In [247]:
df_restaurants.descript.value_counts()

Eating & Drinking                2051
Eating & Drinking w/ Take Out    1633
Retail Food                       838
Mobile Food Walk On               143
Name: descript, dtype: int64

In [299]:
def retrieve_yelp_business_match(row):
    """
    Function to retrieve information from Yelp for the F&B establishment, using Yelp's match endpoint.
    The endpoint takes in the businessname & address and does an approxmiate match to find the estabishment.
    """
    
    response = yelp_api.business_match_query(name=row.businessname,
                                             address1=row.address,
                                             city=row.city,
                                             state='MA',
                                             country='US',
                                             match_threshold=None
                                            )
    
    if not response['businesses']:
        return response
    else:
        if len(response['businesses']) == 1:
            return response['businesses'][0]
        
        else:
            
            print(f"Multiple entries for:\n{row.businessname,row.address}")
            [print(i) for i in response['businesses']]
            n = int(input("Enter row to consider. start number at 1"))
            return response['businesses'][n-1]
        

In [291]:
df_restaurants.loc[:,'yelp_id'] = np.nan
df_restaurants.loc[:,'yelp_alias'] = np.nan
df_restaurants.loc[:,'address1'] = np.nan
df_restaurants.loc[:,'url'] = np.nan

In [1]:
# Loop to get yelp info for each restaurant

for row_index,row_series in df_restaurants.loc[4429:].iterrows():
    
    try:
        result = retrieve_yelp_business_match(row_series)
    except Exception as err:
        print(f"Error Message: {err}")
        input()
        
    
    print(result)
    if result:
        try:
            print(result['id'])
            df_restaurants.loc[row_index,'yelp_id'] = result['id']
        except:
            pass
        try:
            print(result['alias'])
            df_restaurants.loc[row_index,'yelp_alias'] = result['alias']
        except:
            pass
        try:
            print(result['location']['address1'])
            
            df_restaurants.loc[row_index,'address1'] = result['location']['address1']
        except:
            pass
    else:
        nan_counter += 1
    print(row_index,df_restaurants.iloc[row_index,[0,20,28,30]])
    print("-------")
    
    if row_index%25 == 0:
        df_restaurants.to_csv("restaurant_info.csv",index=False)
        
df_restaurants.to_csv("restaurant_info.csv",index=False)

In [330]:
df_restaurants.to_csv("../Data/restaurant_info.csv",index=False)

In [392]:
df_restaurants = pd.read_csv("../Data/restaurant_info.csv")

In [394]:
df_restaurants.yelp_id.isna().sum()

1568

In [395]:
#Dropping these values as Yelp info cannot be obtained on them

df_restaurants_filtered = df_restaurants[~df_restaurants.yelp_id.isna()].reset_index()

In [398]:
df_restaurants_filtered['url'] = np.nan
df_restaurants_filtered['review_count'] = np.nan
df_restaurants_filtered['rating'] = np.nan
df_restaurants_filtered['price'] = np.nan
df_restaurants_filtered['categories'] = np.nan

In [408]:
def retrieve_yelp_business_details(row):
    """
    Retrive business details from each business' page using yelp api's business ID endpoint
    """
    print("before id querying")
    
    response = yelp_api.business_query(id=row)
    print("after id querying")
    return response
        

In [2]:
for row_index,row_series in df_restaurants_filtered.iterrows():
    
    try:
        result = retrieve_yelp_business_details(row_series.yelp_id)
    except Exception as err:
        print(f"Error Message: {err}")
#         input()
        
    
    print(result['id'])
    print(result['url'])
    print(result['alias'])
    if result:
        try:
            print(result['url'])
            df_restaurants_filtered.loc[row_index,'url'] = result['url']
        except:
            pass
        try:
            print(result['review_count'])
            df_restaurants_filtered.loc[row_index,'review_count'] = result['review_count']
        except:
            pass
        try:
            print(result['rating'])
            df_restaurants_filtered.loc[row_index,'rating'] = result['rating']
        except:
            pass
        try:
            print(result['price'])
            df_restaurants_filtered.loc[row_index,'price'] = result['price']
        except:
            pass
        try:
            print(result['categories'])
            df_restaurants_filtered.loc[row_index,'categories'] = result['categories']
        except:
            pass
        
    else:
        nan_counter += 1
    print(row_index,df_restaurants_filtered.iloc[row_index,[0,31,32,33]])
    print("-------")
    
    if row_index%25 == 0:
        df_restaurants_filtered.to_csv("restaurant_info_filtered.csv",index=False)
        
df_restaurants_filtered.to_csv("../Data/restaurant_info_filtered.csv",index=False)

In [400]:
df_restaurants_filtered.to_csv("../Data/restaurant_info_filtered.csv",index=False)

In [415]:
df_restaurants_filtered.shape

(3097, 37)